In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plot_SNVM_data as pSd
import SNVM_B_xyz_extraction as Sdp
from math import ceil

import matplotlib as mpl
from matplotlib_scalebar.scalebar import ScaleBar
from math import ceil,floor,pi
import colorsys
import cv2
from skimage.util.shape import view_as_blocks
from skimage.io import imsave
from skimage.measure import profile_line
import scipy.ndimage

In [ ]:
foldername = '/home/julisko/Seafile/My Library/Mumax/Mumax_Files/DyIG/DyIG_DWtest.out/'
#filename = 'demag_DyIG_test_anisU20k_initial.csv' #512,4,256
#filename = 'demag_DyIG_test_anisU200k_relaxed.csv' #512,4,256
#filename = 'demag_DyIG_test_PBC050_initial.csv' #512,4,256
filename = 'demag_DyIG_test_anisU600k_PBC050_relaxed.csv' #512,4,256
#filename = 'demag_DyIG_test.csv' #512,64,256
pathname  = foldername+filename

foldername = '/home/julisko/Seafile/My Library/Mumax/Mumax_Files/DyIG/DyIG_2DWtest.out/'
filename = 'demag_DyIG_2DW_test.csv' #512,4,256
pathname  = foldername+filename

In [ ]:
#read file

file_df = pd.read_csv(pathname, header=None)
#print(file_df.shape,'\n',file_df)
x_shape = 512
y_shape = 4
z_shape = 256
data = file_df.to_numpy()
data = data.reshape((3,z_shape,y_shape,x_shape))
data_yavg = np.average(data, axis = 2)
#print(data.shape, data)
demag_x = data[0]
demag_y = data[1]
demag_z = data[2]

demag_x_yavg = data_yavg[0]
demag_y_yavg = data_yavg[1]
demag_z_yavg = data_yavg[2]

#demag_x, demag_y, demag_z = np.zeros([x_shape,y_shape,z_shape],dtype = 'float32')
#for x_idx in range(x_shape):
#    for y_idx in range(y_shape):
#        for z_idx in range(z_shape):
#            demag_x = 

In [ ]:
############ Read and extract linecuts from overview image #############

basepath = '/home/julisko/Seafile/My Library/SNVM/20230227_QZabre_Visit/Visit_Feb_2023'
#folder = '/2023-03-04/odmr_fast' #DyIG circle
#folderday = '/2023-03-04-17-16-26-odmr_fast'

folder = '/2023-03-04/odmr_fast' #DyIG overview extended features
folderday = '/2023-03-04-22-43-08-odmr_fast'

#folder = '/2023-03-05/odmr_fast' #DyIG extended features corner
#folderday = '/2023-03-05-10-54-54-odmr_fast'


file,metadata = pSd.ReadSNVMfile(basepath+folder+folderday)

data = pSd.ExtractData(file,datatype = 'odmr:mTesla')
data = pSd.AdjustDataToQS3(data)


#DyIG extended features overview

artif_offset = 0.6

x0_linecut,y0_linecut = 210, 340 #px
angle_linecut = -11.8-90 #deg
length = 120 #px
num = length+1
width = 30

px_shape = metadata['resolution'] #shape in px
actualshape = metadata['rect']['size'] #get shape in m
len_per_px = actualshape[0]/px_shape[0]*1e6 #um/px
linecut_actual_len = length * len_per_px #length of linecut in um

angle_linecut_rad = angle_linecut*2*np.pi/360

x1_linecut = x0_linecut+length*np.sin(angle_linecut_rad+np.pi/2)
y1_linecut = y0_linecut+length*np.cos(angle_linecut_rad+np.pi/2)

fig_ims,ax_ims = plt.subplots(figsize=(20,20))

B_stray_linecut_avg, B_stray_indiv_lcs = pSd.ExtractLinecut(data, metadata, 
                                                            x0_linecut, y0_linecut, x1_linecut, y1_linecut, 
                                                            method = 'map_coordinates', line_width = width, num=num)

ax_ims.imshow(data,cmap='magma')
scalebar = ScaleBar(actualshape[0]/px_shape[0], length_fraction=0.5, 
                    width_fraction=0.02, box_alpha=0.5,frameon=True,box_color='w',location='upper right')
ax_ims.add_artist(scalebar)


fig_lc_avg,ax_lc_avg = plt.subplots()
ax_lc_avg.plot(np.linspace(0,linecut_actual_len,num=len(B_stray_linecut_avg)),B_stray_linecut_avg, color='C0', label = str(np.round(angle_linecut+90,1)))
ax_lc_avg.set_xlabel(r'Distance ($\mu$m)')
#ax_lc_avg.legend()

ax_ims.arrow(x0_linecut,y0_linecut,x1_linecut-x0_linecut,y1_linecut-y0_linecut,color='C0',width = width, 
            head_width = width, head_length = 3)

#fig_lcs,ax_lcs = plt.subplots(figsize=(width,width))
#for linecut_pos in range(width):
#   
#    ax_lc = fig_lcs.add_subplot(ceil(np.sqrt((width))),ceil(np.sqrt((width))),linecut_pos+1)
#    ax_lc.plot(B_stray_indiv_lcs[linecut_pos],label='Raw data profile_line, shift='+str(linecut_pos-(width-1)/2))
#    ax_lc.legend()

x0_linecut,y0_linecut = 342, 230 #px
angle_linecut = 106.77-90 #deg

angle_linecut_rad = angle_linecut*2*np.pi/360

x1_linecut = x0_linecut+length*np.sin(angle_linecut_rad+np.pi/2)
y1_linecut = y0_linecut+length*np.cos(angle_linecut_rad+np.pi/2)

B_stray_linecut_avg, B_stray_indiv_lcs = pSd.ExtractLinecut(data, metadata, 
                                                            x0_linecut, y0_linecut, x1_linecut, y1_linecut, 
                                                            method = 'map_coordinates', line_width = width, num=num)
ax_lc_avg.plot(np.linspace(0,linecut_actual_len,num=len(B_stray_linecut_avg)),B_stray_linecut_avg+artif_offset, color='C1', label = str(np.round(angle_linecut+90,1)))
ax_ims.arrow(x0_linecut,y0_linecut,x1_linecut-x0_linecut,y1_linecut-y0_linecut,width = width, 
            head_width = width, head_length = 3, color='C1')


x0_linecut,y0_linecut = 230, 270 #px
angle_linecut = -144.42-90 #deg

angle_linecut_rad = angle_linecut*2*np.pi/360

x1_linecut = x0_linecut+length*np.sin(angle_linecut_rad+np.pi/2)
y1_linecut = y0_linecut+length*np.cos(angle_linecut_rad+np.pi/2)

B_stray_linecut_avg, B_stray_indiv_lcs = pSd.ExtractLinecut(data, metadata, 
                                                            x0_linecut, y0_linecut, x1_linecut, y1_linecut, 
                                                            method = 'map_coordinates', line_width = width, num=num)
ax_lc_avg.plot(np.linspace(0,linecut_actual_len,num=len(B_stray_linecut_avg)),B_stray_linecut_avg+2*artif_offset, color='C2', label = str(np.round(angle_linecut+90,1)))
ax_ims.arrow(x0_linecut,y0_linecut,x1_linecut-x0_linecut,y1_linecut-y0_linecut,width = width, 
            head_width = width, head_length = 3, color='C2')


x0_linecut,y0_linecut = 390, 290 #px
angle_linecut = 21.62-90 #deg
width = 20

angle_linecut_rad = angle_linecut*2*np.pi/360

x1_linecut = x0_linecut+length*np.sin(angle_linecut_rad+np.pi/2)
y1_linecut = y0_linecut+length*np.cos(angle_linecut_rad+np.pi/2)

B_stray_linecut_avg, B_stray_indiv_lcs = pSd.ExtractLinecut(data, metadata, 
                                                            x0_linecut, y0_linecut, x1_linecut, y1_linecut, 
                                                            method = 'map_coordinates', line_width = width, num=num)
ax_lc_avg.plot(np.linspace(0,linecut_actual_len,num=len(B_stray_linecut_avg)),B_stray_linecut_avg+3*artif_offset, color='C3', label = str(np.round(angle_linecut+90,1)))
ax_ims.arrow(x0_linecut,y0_linecut,x1_linecut-x0_linecut,y1_linecut-y0_linecut,width = width, 
            head_width = width, head_length = 3, color='C3')


x0_linecut,y0_linecut = 170, 340 #px
angle_linecut = -100.29-90 #deg
width = 15

angle_linecut_rad = angle_linecut*2*np.pi/360

x1_linecut = x0_linecut+length*np.sin(angle_linecut_rad+np.pi/2)
y1_linecut = y0_linecut+length*np.cos(angle_linecut_rad+np.pi/2)

B_stray_linecut_avg, B_stray_indiv_lcs = pSd.ExtractLinecut(data, metadata, 
                                                            x0_linecut, y0_linecut, x1_linecut, y1_linecut, 
                                                            method = 'map_coordinates', line_width = width, num=num)
ax_lc_avg.plot(np.linspace(0,linecut_actual_len,num=len(B_stray_linecut_avg)),B_stray_linecut_avg+4.5*artif_offset, color='C4', label = str(np.round(angle_linecut+90,1)))
ax_ims.arrow(x0_linecut,y0_linecut,x1_linecut-x0_linecut,y1_linecut-y0_linecut,width = width, 
            head_width = width, head_length = 3, color='C4')


x0_NV,y0_NV = 50, 50 #px
angle_NV = 144.75 #deg
angle_NV_rad = angle_NV*2*np.pi/360
x1_NV = x0_NV+30*np.sin(angle_NV_rad+np.pi/2)
y1_NV = y0_NV+30*np.cos(angle_NV_rad+np.pi/2)
ax_ims.arrow(x0_NV,y0_NV,x1_NV-x0_NV,y1_NV-y0_NV,width = 2, color='k')

fig_lc_avg.legend()
ax_lc_avg.set_ylabel('Stray Field (mT)')

In [ ]:
foldername = '/home/julisko/Seafile/My Library/Mumax/Mumax_Files/DyIG/DyIG_2DWtest.out/'
filename = 'demag_DyIG_2DW_test.csv' #512,4,256
pathname  = foldername+filename

#read file
file_df = pd.read_csv(pathname, header=None)
x_shape = 512
y_shape = 4
z_shape = 256
data = file_df.to_numpy()
data = data.reshape((3,z_shape,y_shape,x_shape))
demag_x = data[0]
demag_y = data[1]
demag_z = data[2]

strength = np.sqrt(demag_x**2+demag_y**2+demag_y**2)

# plot data
liftheight = 89

im = plt.imshow(demag_x[::-1,int(y_shape/2),:])#,vmin = -0.0001, vmax = 0.0001)
cb=plt.colorbar(im)

In [ ]:
#project onto NV axis

theta_NV = 90
phi_NV = 144.75
phi_DWpara = -11.8  #angle_linecut = -11.8-90 #deg
#phi_DWpara = 106.77  #angle_linecut = 106.77-90 #deg
#phi_DWpara = -144.42  #angle_linecut = -144.42-90 #deg
#phi_DWpara = 21.62  #angle_linecut = 21.62-90 #deg
#phi_DWpara = -100.29  #angle_linecut = -100.29-90 #deg


phi_DWperp = phi_DWpara - 90

theta_NVrot = theta_NV
phi_NVrot = 180 + phi_NV - phi_DWperp
#phi_NVrot = phi_NV - phi_DWperp

theta_NVrot_rad = np.radians(theta_NVrot)
phi_NVrot_rad = np.radians(phi_NVrot)

demag_proj = demag_x * np.sin(theta_NVrot_rad)*np.cos(phi_NVrot_rad) + demag_y * np.sin(theta_NVrot_rad) * np.sin(phi_NVrot_rad) + demag_z * np.cos(theta_NVrot_rad)
demag_proj_yavg = demag_x_yavg * np.sin(theta_NVrot_rad)*np.cos(phi_NVrot_rad) + demag_y_yavg * np.sin(theta_NVrot_rad) * np.sin(phi_NVrot_rad) + demag_z_yavg * np.cos(theta_NVrot_rad)


print(theta_NVrot, phi_NVrot)


###### plot data

liftheight = 139

plt.imshow(demag_x[liftheight,:,:])
plt.imshow(demag_x[::-1,int(y_shape/2),:])
plt.show()
plt.plot(demag_x[liftheight,int(y_shape/2),:])
plt.show()
plt.imshow(demag_proj[liftheight,:,:])
fig,ax=plt.subplots()
im = ax.imshow(demag_proj[::-1,int(y_shape/2),:])
cb=fig.colorbar(im)
plt.show()
plt.plot(demag_proj[liftheight,int(y_shape/2),:])
plt.plot(demag_proj_yavg[liftheight,:])
plt.show()
plt.imshow(demag_proj_yavg[::-1,:])


In [ ]:
#####  project multiple DW types onto all different DW directions (linecut directions)

foldername = '/home/julisko/Seafile/My Library/Mumax/Mumax_Files/DyIG/DyIG_2DWtest.out/'
filenames = ['demag_DyIG_2DW_010_0-10.csv', #512,4,256
             'demag_DyIG_2DW_0-10_010.csv',
             'demag_DyIG_2DW_001_00-1.csv',
             'demag_DyIG_2DW_00-1_001.csv',
             'demag_DyIG_2DW_010_010.csv',
             'demag_DyIG_2DW_0-10_010_anisU10.50.csv']
pathnames  = [foldername+filenames[i] for i in range(len(filenames))]
#print((pathnames))

x_shape = 512
y_shape = 4
z_shape = 256

x_scaling = 10
cellsize = 1e-9

cutheight = 139

theta_NV = 90
phi_NV = 144.75

phis_DWpara = [-11.8,
              106.77,
              -144.42,
              21.62,
              -100.29]

artif_offset = 6e-4

#read files
files_df = [[]]*len(filenames)
multiple_data = [[]]*len(filenames)

multiple_demag_x = [[]]*len(filenames)
multiple_demag_y = [[]]*len(filenames)
multiple_demag_z = [[]]*len(filenames)

multiple_demag_proj = [[[]]*len(filenames)]*len(phis_DWpara)

#print(np.shape(multiple_data),np.shape(multiple_demag_x),np.shape(multiple_demag_y),np.shape(multiple_demag_z), np.shape(multiple_demag_proj))

# read all specified files

for file_ind in range(len(filenames)):
    files_df[file_ind] = pd.read_csv(pathnames[file_ind], header=None)

    multiple_data[file_ind] = files_df[file_ind].to_numpy()
    multiple_data[file_ind] = multiple_data[file_ind].reshape((3,z_shape,y_shape,x_shape))
    multiple_demag_x[file_ind] = multiple_data[file_ind][0]
    multiple_demag_y[file_ind] = multiple_data[file_ind][1]
    multiple_demag_z[file_ind] = multiple_data[file_ind][2]
    
    
    
    #fig_file,  ax_file = plt.subplots(figsize = (25,15))
    fig_file = plt.figure(figsize = (25,15))
    #ax_file.get_xaxis().set_visible(False)
    #ax_file.get_yaxis().set_visible(False)
    fig_file.suptitle(filenames[file_ind])
    
    fig_file_stacked, ax_file_stacked = plt.subplots(figsize = (7,7))
    ax_file_stacked.set_title(filenames[file_ind])
    
    # project onto all specified angles
    
    for phi_ind in range(len(phis_DWpara)):
        
        phi_DWperp = phis_DWpara[phi_ind] - 90
        
        theta_NVrot = theta_NV
        phi_NVrot = 180 + phi_NV - phi_DWperp
        
        theta_NVrot_rad = np.radians(theta_NVrot)
        phi_NVrot_rad = np.radians(phi_NVrot)
        
        multiple_demag_proj[phi_ind][file_ind] = multiple_demag_x[file_ind] * np.sin(theta_NVrot_rad)*np.cos(phi_NVrot_rad) + multiple_demag_y[file_ind] * np.sin(theta_NVrot_rad) * np.sin(phi_NVrot_rad) + multiple_demag_z[file_ind] * np.cos(theta_NVrot_rad)
        
        #print(file_ind, phi_ind, phi_DWperp, multiple_demag_proj[phi_ind][file_ind][cutheight,int(y_shape/2),256])
        
        #plt.plot(multiple_demag_proj[phi_ind][file_ind][cutheight,int(y_shape/2),:])
        #plt.plot(multiple_demag_z[file_ind][cutheight,int(y_shape/2),:])
        #plt.show()
        
        
        
        ax_phi = fig_file.add_subplot(ceil(np.sqrt((len(phis_DWpara)))),ceil(np.sqrt((len(phis_DWpara)))),phi_ind+1)
        ax_phi.plot(np.linspace(0,x_scaling*cellsize*x_shape,x_shape),
                    multiple_demag_proj[phi_ind][file_ind][cutheight,int(y_shape/2),:])
        ax_file_stacked.plot(np.linspace(0,x_scaling*cellsize*x_shape,x_shape)*1e6,
                             (multiple_demag_proj[phi_ind][file_ind][cutheight,int(y_shape/2),:]+phi_ind*artif_offset)*1e3,
                            label = str(np.round(phis_DWpara[phi_ind],1)))
        ax_file_stacked.set_xlabel('Distance (µm)')
        ax_file_stacked.set_ylabel('Demag Field (T)')
        #ax_phi.imshow(multiple_demag_proj[phi_ind][file_ind][::-1,int(y_shape/2),:])
        ax_phi.set_title(str(phis_DWpara[phi_ind]))
        #print(str(phis_DWpara[phi_ind]))
#    fig_file_stacked.legend()
        

### plotting !!!!! does not work this way, apparently all demag for projection angles are saved as the last one !!!!!!


#for file_ind in range(len(filenames)):    
#    
#    #fig_file,  ax_file = plt.subplots(figsize = (25,15))
#    fig_file = plt.figure(figsize = (25,15))
#    #ax_file.get_xaxis().set_visible(False)
#    #ax_file.get_yaxis().set_visible(False)
#    fig_file.suptitle(filenames[file_ind])
#    
#    for phi_ind in range(len(phis_DWpara)):
#                
#        ax_phi = fig_file.add_subplot(ceil(np.sqrt((len(phis_DWpara)))),ceil(np.sqrt((len(phis_DWpara)))),phi_ind+1)
#        ax_phi.plot(np.linspace(0,x_scaling*cellsize*x_shape,x_shape),
#                    multiple_demag_proj[phi_ind][file_ind][cutheight,int(y_shape/2),:])
#        #ax_phi.imshow(multiple_demag_proj[phi_ind][file_ind][::-1,int(y_shape/2),:])
#        ax_phi.set_title(str(phis_DWpara[phi_ind]))
#        #print(str(phis_DWpara[phi_ind]))
       


In [ ]:
#####  use anisU direction-adjusted simulations #########
####  DW type fixed #####

fixedDWtype_anisUadj_foldername = '/home/julisko/Seafile/My Library/Mumax/Mumax_Files/DyIG/DyIG_2DW_adjAnisU_ODMR2023-03-04-22-43-08.out/'
fixedDWtype_anisUadj_filenames = ['demag_DyIG_2DW_overview_DW0.csv', #512,4,256
                                  'demag_DyIG_2DW_overview_DW1.csv',
                                  'demag_DyIG_2DW_overview_DW2.csv',
                                  'demag_DyIG_2DW_overview_DW3.csv',
                                  'demag_DyIG_2DW_overview_DW4.csv']
fixedDWtype_anisUadj_pathnames  = [fixedDWtype_anisUadj_foldername+fixedDWtype_anisUadj_filenames[i] 
                                    for i in range(len(fixedDWtype_anisUadj_filenames))]
#print((pathnames))

x_shape = 512
y_shape = 4
z_shape = 256

x_scaling = 10
cellsize = 1e-9

cutheight = 139

theta_NV = 90
phi_NV = 144.75

phis_DWpara = [-11.8, #DW0
              106.77, #DW1
              -144.42, #DW2
              21.62, #DW3
              -100.29] #DW4

artif_offset = 0 #2e-4

#read files
fixedDWtype_anisUadj_files_df = [[]]*len(fixedDWtype_anisUadj_filenames)
fixedDWtype_anisUadj_multiple_data = [[]]*len(fixedDWtype_anisUadj_filenames)

fixedDWtype_anisUadj_multiple_demag_x = [[]]*len(fixedDWtype_anisUadj_filenames)
fixedDWtype_anisUadj_multiple_demag_y = [[]]*len(fixedDWtype_anisUadj_filenames)
fixedDWtype_anisUadj_multiple_demag_z = [[]]*len(fixedDWtype_anisUadj_filenames)

fixedDWtype_anisUadj_multiple_demag_proj = [[]]*len(fixedDWtype_anisUadj_filenames)

#print(np.shape(multiple_data),np.shape(multiple_demag_x),np.shape(multiple_demag_y),np.shape(multiple_demag_z), np.shape(multiple_demag_proj))

# read all specified files

fig_file_stacked, ax_file_stacked = plt.subplots(figsize = (7,7))
ax_file_stacked.set_title('Adjusted anisU direction righthand IP DW')

for DW_ind in range(len(fixedDWtype_anisUadj_filenames)):
    fixedDWtype_anisUadj_files_df[DW_ind] = pd.read_csv(fixedDWtype_anisUadj_pathnames[DW_ind], header=None)

    fixedDWtype_anisUadj_multiple_data[DW_ind] = fixedDWtype_anisUadj_files_df[DW_ind].to_numpy()
    fixedDWtype_anisUadj_multiple_data[DW_ind] = fixedDWtype_anisUadj_multiple_data[DW_ind].reshape((3,z_shape,y_shape,x_shape))
    fixedDWtype_anisUadj_multiple_demag_x[DW_ind] = fixedDWtype_anisUadj_multiple_data[DW_ind][0]
    fixedDWtype_anisUadj_multiple_demag_y[DW_ind] = fixedDWtype_anisUadj_multiple_data[DW_ind][1]
    fixedDWtype_anisUadj_multiple_demag_z[DW_ind] = fixedDWtype_anisUadj_multiple_data[DW_ind][2]
    
    
    # project onto DW-specific angle
        
    phi_DWperp = phis_DWpara[DW_ind] - 90

    theta_NVrot = theta_NV
    phi_NVrot = 180 + phi_NV - phi_DWperp

    theta_NVrot_rad = np.radians(theta_NVrot)
    phi_NVrot_rad = np.radians(phi_NVrot)

    fixedDWtype_anisUadj_multiple_demag_proj[DW_ind] = fixedDWtype_anisUadj_multiple_demag_x[DW_ind] * np.sin(theta_NVrot_rad)*np.cos(phi_NVrot_rad) + fixedDWtype_anisUadj_multiple_demag_y[DW_ind] * np.sin(theta_NVrot_rad) * np.sin(phi_NVrot_rad) + fixedDWtype_anisUadj_multiple_demag_z[DW_ind] * np.cos(theta_NVrot_rad)

    #print(file_ind, phi_ind, phi_DWperp, multiple_demag_proj[phi_ind][file_ind][cutheight,int(y_shape/2),256])

    #plt.plot(multiple_demag_proj[phi_ind][file_ind][cutheight,int(y_shape/2),:])
    #plt.plot(multiple_demag_z[file_ind][cutheight,int(y_shape/2),:])
    #plt.show()

    ax_file_stacked.plot(np.linspace(0,x_scaling*cellsize*x_shape,x_shape)*1e6,
                         (fixedDWtype_anisUadj_multiple_demag_proj[DW_ind][cutheight,int(y_shape/2),:]
                          +DW_ind*artif_offset)*1e3,
                        label = str(np.round(phis_DWpara[DW_ind],1)))
    ax_file_stacked.set_xlabel('Distance (µm)')
    ax_file_stacked.set_ylabel('Demag Field (T)')
    #ax_phi.imshow(multiple_demag_proj[phi_ind][file_ind][::-1,int(y_shape/2),:])
    #print(str(phis_DWpara[phi_ind]))
    fig_file_stacked.legend()
        

### plotting !!!!! does not work this way, apparently all demag for projection angles are saved as the last one !!!!!!


#for file_ind in range(len(filenames)):    
#    
#    #fig_file,  ax_file = plt.subplots(figsize = (25,15))
#    fig_file = plt.figure(figsize = (25,15))
#    #ax_file.get_xaxis().set_visible(False)
#    #ax_file.get_yaxis().set_visible(False)
#    fig_file.suptitle(filenames[file_ind])
#    
#    for phi_ind in range(len(phis_DWpara)):
#                
#        ax_phi = fig_file.add_subplot(ceil(np.sqrt((len(phis_DWpara)))),ceil(np.sqrt((len(phis_DWpara)))),phi_ind+1)
#        ax_phi.plot(np.linspace(0,x_scaling*cellsize*x_shape,x_shape),
#                    multiple_demag_proj[phi_ind][file_ind][cutheight,int(y_shape/2),:])
#        #ax_phi.imshow(multiple_demag_proj[phi_ind][file_ind][::-1,int(y_shape/2),:])
#        ax_phi.set_title(str(phis_DWpara[phi_ind]))
#        #print(str(phis_DWpara[phi_ind]))
       


In [ ]:
### plot geometry

geom_foldername = '/home/julisko/Seafile/My Library/Mumax/Mumax_Files/DyIG/DyIG_2DWtest.out/'
geom_filename = 'DyIG_2DWtest_geom.csv' #512,4,256
#geom_filename = 'regions000000.csv' #512,4,256
geom_pathname  = geom_foldername+geom_filename

geomfile_df = pd.read_csv(geom_pathname, header=None)
#print(file_df.shape,'\n',file_df)
x_shape = 512
y_shape = 4
z_shape = 256
geom_data = geomfile_df.to_numpy()
print(np.shape(geom_data))
geom_data = geom_data.reshape((z_shape,y_shape,x_shape))
#print(data.shape, data)
geom = geom_data

fig,ax=plt.subplots(figsize=(15,15))
im = ax.imshow(geom[:,int(y_shape/2),:])
#cb=fig.colorbar(im)
plt.show()

In [ ]:
basepath = '/home/julisko/Seafile/My Library/SNVM/20230227_QZabre_Visit/Visit_Feb_2023'
#folder = '/2023-03-04/odmr_fast' #DyIG circle
#folderday = '/2023-03-04-17-16-26-odmr_fast'

folder = '/2023-03-04/odmr_fast' #DyIG overview extended features
folderday = '/2023-03-04-22-43-08-odmr_fast'

#folder = '/2023-03-05/odmr_fast' #DyIG extended features corner
#folderday = '/2023-03-05-10-54-54-odmr_fast'


file,metadata = pSd.ReadSNVMfile(basepath+folder+folderday)

data = pSd.ExtractData(file,datatype = 'odmr:mTesla')
data = pSd.AdjustDataToQS3(data)
fig, ax, im, cb, scalebar = pSd.PlotData(data, metadata, datalabel='Stray Field')#,minval=0.55,maxval=0.8)#,backgroundcorr='y')

ax.get_xaxis().set_visible(True)
ax.get_yaxis().set_visible(True)


#### plot angles

x0_NV,y0_NV = 70, 70 #px
angle_NV = phi_NV #deg
angle_NV_rad = angle_NV*2*np.pi/360
x1_NV = x0_NV+40*np.sin(angle_NV_rad+np.pi/2)
y1_NV = y0_NV+40*np.cos(angle_NV_rad+np.pi/2)
ax.arrow(x0_NV,y0_NV,x1_NV-x0_NV,y1_NV-y0_NV,width = 5, color='C0')

x0_proj,y0_proj = 70,70 #px
angle_proj = phi_proj #deg
angle_proj_rad = angle_proj*2*np.pi/360
x1_proj = x0_proj+40*np.sin(angle_proj_rad+np.pi/2)
y1_proj = y0_proj+40*np.cos(angle_proj_rad+np.pi/2)
ax.arrow(x0_proj,y0_proj,x1_proj-x0_proj,y1_proj-y0_proj,width = 5, color='C1')

x0_DWperp,y0_DWperp = 70,70 #px
angle_DWperp = phi_DWperp #deg
angle_DWperp_rad = angle_DWperp*2*np.pi/360
x1_DWperp = x0_DWperp+40*np.sin(angle_DWperp_rad+np.pi/2)
y1_DWperp = y0_DWperp+40*np.cos(angle_DWperp_rad+np.pi/2)
ax.arrow(x0_DWperp,y0_DWperp,x1_DWperp-x0_DWperp,y1_DWperp-y0_DWperp,width = 5, color='C2')

phi_NVrot = 180 + phi_NV - angle_DWperp
x0_NVrot,y0_NVrot = 70,70 #px
angle_NVrot = phi_NVrot #deg
angle_NVrot_rad = angle_NVrot*2*np.pi/360
x1_NVrot = x0_NVrot+40*np.sin(angle_NVrot_rad+np.pi/2)
y1_NVrot = y0_NVrot+40*np.cos(angle_NVrot_rad+np.pi/2)
ax.arrow(x0_NVrot,y0_NVrot,x1_NVrot-x0_NVrot,y1_NVrot-y0_NVrot,width = 5, color='C3')

plt.show()

In [ ]:
# !! with anisU in direction (1,0.5,0) !!


#project onto NV axis

theta_NV = 90
phi_NV = 144.75
phi_DWpara = -11.8  #angle_linecut = -11.8-90 #deg
#phi_DWpara = 106.77  #angle_linecut = 106.77-90 #deg
#phi_DWpara = -144.42  #angle_linecut = -144.42-90 #deg
#phi_DWpara = 21.62  #angle_linecut = 21.62-90 #deg
#phi_DWpara = -100.29  #angle_linecut = -100.29-90 #deg


phi_DWperp = phi_DWpara - 90

theta_NVrot = theta_NV
phi_NVrot = 180 + phi_NV - phi_DWperp
#phi_NVrot = phi_NV - phi_DWperp

theta_NVrot_rad = np.radians(theta_NVrot)
phi_NVrot_rad = np.radians(phi_NVrot)

demag_proj = demag_x * np.sin(theta_NVrot_rad)*np.cos(phi_NVrot+np.pi/2) + demag_y * np.sin(theta_NVrot_rad) * np.sin(phi_NVrot+np.pi/2) + demag_z * np.cos(theta_NVrot_rad)
demag_proj_yavg = demag_x_yavg * np.sin(theta_NVrot_rad)*np.cos(phi_NVrot+np.pi/2) + demag_y_yavg * np.sin(theta_NVrot_rad) * np.sin(phi_NVrot+np.pi/2) + demag_z_yavg * np.cos(theta_NVrot_rad)


print(theta_NVrot, phi_NVrot)


###### plot data

liftheight = 169

plt.imshow(demag_x[liftheight,:,:])
plt.imshow(demag_x[::-1,int(y_shape/2),:])
plt.show()
plt.plot(demag_x[liftheight,int(y_shape/2),:])
plt.show()
plt.imshow(demag_proj[liftheight,:,:])
fig,ax=plt.subplots()
im = ax.imshow(demag_proj[::-1,int(y_shape/2),:])
cb=fig.colorbar(im)
plt.show()
plt.plot(demag_proj[liftheight,int(y_shape/2),:])
#plt.plot(demag_proj_yavg[liftheight,:])
plt.show()
#plt.imshow(demag_proj_yavg[::-1,:])
